In [2]:
# #########################
# 
# 
# BE CAREFUL: To use this program, you need to own the Diagnostic Pack license for your database
#
#
# #########################


import cx_Oracle
import plotly as py
import plotly.graph_objs as go
import numpy as np

# Database Connection
#con = cx_Oracle.connect('hellsulf/vOJ1m04e@pd01-scan.prod.receptpartner.se:1521/SBPGD_PRIMARY')
con = cx_Oracle.connect('hellsulf/vOJ1m04e@pd02-scan.prod.receptpartner.se:1521/SBPRR_PRIMARY')

# Query
q='with t as (' \
'select to_char(mtime,\'YYYY/MM/DD\') mtime, to_char(mtime,\'HH24\') d, load AAS from ( '\
'select to_date(mtime,\'YYYY/MM/DD HH24\') mtime, '\
'round(sum(c1),2) AAS_WAIT, '\
'round(sum(c2),2) AAS_CPU, '\
'round(sum(cnt),2) AAS, '\
'round(sum(load),2) LOAD '\
'from ( '\
'select to_char(sample_time,\'YYYY-MM-DD HH24\') mtime, '\
'decode(session_state,\'WAITING\',count(*),0)/360 c1, '\
'decode(session_state,\'ON CPU\',count(*),0) /360 c2, '\
'count(*)/360 cnt, '\
'count(*)/360/cpu.core_nb load '\
' from dba_hist_active_sess_history '\
' ,(select value as core_nb from v$osstat where stat_name=\'NUM_CPU_CORES\') cpu '\
' where sample_time>sysdate-365 '\
' group by to_char(sample_time,\'YYYY-MM-DD HH24\'),session_state, cpu.core_nb '\
' ) '\
'group by mtime '\
'    ) '\
') '\
'select mtime, '\
'nvl("00-01_ ",0) , '\
'nvl("01-02_ ",0) , '\
'nvl("02-03_ ",0) , '\
'nvl("03-04_ ",0) , '\
'nvl("04-05_ ",0) , '\
'nvl("05-06_ ",0) , '\
'nvl("06-07_ ",0) , '\
'nvl("07-08_ ",0) , '\
'nvl("08-09_ ",0) , '\
'nvl("09-10_ ",0) , '\
'nvl("10-11_ ",0) , '\
'nvl("11-12_ ",0) , '\
'nvl("12-13_ ",0) , '\
'nvl("13-14_ ",0) , '\
'nvl("14-15_ ",0) , '\
'nvl("15-16_ ",0) , '\
'nvl("16-17_ ",0) , '\
'nvl("17-18_ ",0) , '\
'nvl("18-19_ ",0) , '\
'nvl("19-20_ ",0) , '\
'nvl("20-21_ ",0) , '\
'nvl("21-22_ ",0) , '\
'nvl("22-23_ ",0) , '\
'nvl("23-24_ ",0) '\
'from t '\
'pivot( '\
'  sum(aas) as " " for d in (\'00\' AS "00-01",\'01\' AS "01-02",\'02\' AS "02-03",\'03\' AS "03-04",\'04\' AS "04-05",\'05\' AS "05-06",\'06\' AS "06-07",\'07\' AS "07-08",\'08\' AS "08-09",\'09\' AS "09-10",\'10\' AS "10-11", '\
' \'11\' AS "11-12",\'12\' AS "12-13", \'13\' AS "13-14",\'14\' AS "14-15",\'15\' AS "15-16",\'16\' AS "16-17",\'17\' AS "17-18",\'18\' AS "18-19",\'19\' AS "19-20",\'20\' AS "20-21",\'21\' AS "21-22", '\
' \'22\' AS "22-23",\'23\' AS "23-24") '\
') '\
'order by mtime '

# Now lets create a two dimensional array with the cursor content. 
cur = con.cursor()
cur.execute(q)
# We use a numpy array, much more easy to manipulate.
plt_z = np.array(cur.fetchall())

# First column extraction, to build the Y axis
plt_y=plt_z[:,0]

# Heatmap Data structure 
# x : X axis values
# y : Y axis values
# z : data serie, without the first column (use of delete function to do that) 
data=[{
  'z' : np.delete(plt_z,0,1),
  'x' : ['00-01h','01-02h','02-03h','03-04h','04-05h','05-06h','06-07h','07-08h','08-09h','09-10h','10-11h','11-12h','12-13h','13-14h','14-15h','15-16h','16-17h','17-18h','18-19h','19-20h','20-21h','21-22h','22-23h','23-00h'],
  'y' : plt_y,
  'type' : 'heatmap',
  'colorscale' : [
     [0, 'rgb(240,240,240)'],
     [0.125, 'rgb(198,255,198)'],
     [0.25, 'rgb(61,204,0)'],
     [0.375, 'rgb(255,255,0)'],
     [0.5, 'rgb(255, 216, 0)'],
     [1, 'rgb(255, 0, 0)']
  ],
  'colorbar': {
        'ticks': 'inside'
    },
  'xgap' : 1,
  'ygap' : 1,
  'zmin' : 0, 'zmax' : 2
  }];

# Layout Definition (mainly for Title)
layout=go.Layout(title='DATABASE LOAD HEATMAP = '+con.dsn)

# Figure structure definition with the heatmap data and the layout def.
fig=go.Figure(data=data, layout=layout)

# Display the heatmap and generate and html file for offline visualization.
py.offline.plot( fig, filename='/var/tmp/test.html')

# Database stuff closing
cur.close()
con.close() 